### Установка необходимых пакетов
Читаем из файла `requirements.txt`

**Библиотеки:**
* PyTorch v2.1.0
* Sentencepiece v0.1.99
* Transformers v4.34.1
* Pandas  v2.1.1
* NumPy v1.26.1
* Scikit Learn v1.3.2


In [ ]:
%pip install -r requirements.txt

### Считываем датасет

Файлы берём из папки `/dataset`

Для обучения - `train.csv`

Для валидации - `validate.csv`

Для тестирования - `test.csv`

In [47]:
import pandas as pd # Импортируем библиотеку Pandas

# Отключаем предупреждения
no_deprecation_warning=True

In [38]:
train_data = pd.read_csv('./dataset/train.csv')     # Считываем данные для обучения
valid_data = pd.read_csv('./dataset/validate.csv')  # Считываем данные для валидации

### Инициализируем библиотеку BERT (`rubert-tiny`)

Импортируем необходимые модули

In [39]:
from bert_classifier import BertClassifier  # Импортируем основной класс BertClassifier

Инициализируем `BertClassifier` на `41` группу и `60` эпох

In [40]:
classifier = BertClassifier(
    model_path='cointegrated/rubert-tiny',          # Путь до репозитория с дообучаемой моделью
    tokenizer_path='cointegrated/rubert-tiny',      # Путь до токенизатора
    n_classes=41,                                   # Количество классов для обучения
    epochs=60,                                      # Количество эпох обучения
    max_len=1024,                                   # Максимальный размер текста
    model_save_path='./output/model.pt'             # Путь до сохранения модели
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Подготавливаем данные для обучения и валидации

In [41]:
classifier.preparation(
    X_train=list(train_data['groups']),             # Обучающие поля таблицы с текстом
    y_train=list(train_data['code']),               # Обучающие поля таблицы с реальными кодами групп
    X_valid=list(valid_data['groups']),             # Валидирующие поля таблицы с текстом
    y_valid=list(valid_data['code'])                # Валидирующие поля таблицы с реальными кодами групп
)

c:\Users\mrdas\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Проверяем возможность обучения нейросети на CUDA ядрах и выводим дополнительную информацию

In [48]:
import torch
print(f"CUDA поддерживается этой системой: {torch.cuda.is_available()}")
print(f"Версия CUDA: {torch.version.cuda}")
 
# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(f"ID текущего CUDA устройства: {torch.cuda.current_device()}")
       
print(f"Имя текущего CUDA устройства: {torch.cuda.get_device_name(cuda_id)}")

CUDA поддерживается этой системой: True
Версия CUDA: 11.8
ID текущего CUDA устройства: 0
Имя текущего CUDA устройства: NVIDIA GeForce RTX 4070 Ti


### Обучаем нейросеть

In [43]:
classifier.train()

Эпоха 1/60
Обучение: потери 3.4729375537012666 | точность 0.06205673758865248
Валидация: потери 3.391901946836902 | точность 0.08713692946058091
----------
Эпоха 2/60
Обучение: потери 3.2672960019447435 | точность 0.1427304964539007
Валидация: потери 3.165799225530317 | точность 0.19294605809128632
----------
Эпоха 3/60
Обучение: потери 3.006519720587932 | точность 0.2278368794326241
Валидация: потери 2.9954433902617423 | точность 0.23029045643153526
----------
Эпоха 4/60
Обучение: потери 2.7837150365533962 | точность 0.2783687943262411
Валидация: потери 2.9272385951011413 | точность 0.23858921161825727
----------
Эпоха 5/60
Обучение: потери 2.5900427573163745 | точность 0.33156028368794327
Валидация: потери 2.845012257176061 | точность 0.26141078838174275
----------
Эпоха 6/60
Обучение: потери 2.4319404632272854 | точность 0.3670212765957447
Валидация: потери 2.8227072454267934 | точность 0.27800829875518673
----------
Эпоха 7/60
Обучение: потери 2.3012352426287155 | точность 0.415780

### Тестируем работу обученной модели

In [50]:
import time

test_data  = pd.read_csv('./dataset/test.csv')

texts = list(test_data['groups'])
labels = list(test_data['code'])

predictions = [classifier.predict(t) for t in texts]

report = []     # form report variable

for t, l, p in zip(texts, labels, predictions):
    report.append({
        "real_group": l,
        "predicted_group": p ,
        "isEqual": ("no", "yes")[l == p],
        "groups_set": t
    })

# Экспортируем результаты в CSV файл, поддерживаемый Excel
report_df = pd.DataFrame(report)
report_df.to_csv(f"./reports/report{time.time()}.csv", index_label="id", sep=";", encoding="cp1251")

C:\Users\mrdas\AppData\Local\Temp\ipykernel_23176\3271279555.py:18: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  "isEqual": ("no", "yes")[l == p],


Выводим таблицу с результатами для сверки

In [45]:
report_df.head()

,real_group,predicted_group,isEqual,groups_set
0,1,9,no,ежик в матане околошкольная математика | борис...
1,1,1,yes,physics.math.code высшая математика – просто и...
2,1,1,yes,physics.math.code справедливые мемы про матешу...
3,1,9,no,околошкольная математика | борис трушин physic...
4,1,1,yes,типичный математик code blog physics.math.code...


### Получаем метрики

In [46]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1score = precision_recall_fscore_support(labels, predictions,average='macro', zero_division=0)[:3]

print(f'precision: {precision}, recall: {recall}, f1score: {f1score}')

precision: 0.09115523353853884, recall: 0.1619047619047619, f1score: 0.1021623216699943
